In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget.neurons import adex
from voltagebudget.util import read_args
from voltagebudget.util import read_stim
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import mad
from voltagebudget.util import write_spikes
from voltagebudget.util import filter_voltages

Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


In [9]:
# ---------------------------------------------------------------
stim = "../data/stim1.csv"
verbose = True

# ---------------------------------------------------------------
if verbose:
    print(">>> Importing stimulus from {}.".format(stim))

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

# ---------------------------------------------------------------
f = 20
N = 1
t = 0.4
time_step = 1e-5
seed_value = 42

mode = 'regular'
params, w_in, bias_in, sigma = util.read_modes(mode)

>>> Importing stimulus from ../data/stim1.csv.


In [19]:
A_ref = 0.05e-9
f_ref = 20
ns_ref, ts_ref, voltages_ref = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.001,
    n_cycles=0,
    f=f_ref,
    A=A_ref,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [50]:
A_slow = 0.05e-9
f_slow = 4
ns_slow, ts_slow, voltages_slow = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.2 - .25,
    n_cycles=10,
    f=f_slow,
    A=A_slow,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [51]:
A_fast = 0.05e-9
f_fast = 30
ns_fast, ts_fast, voltages_fast = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.2 - .05,
    n_cycles=10,
    f=f_fast,
    A=A_fast,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [52]:
p = figure(plot_width=400, plot_height=200)
p.vbar(x=ts_ref, width=0.00005, 
       bottom=np.zeros_like(ts_ref), top=np.ones_like(ts_ref), 
       color="black")
p.xaxis.axis_label = 'Times (s)'
p.yaxis.axis_label = 'Spikes'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

p = figure(plot_width=400, plot_height=200)
p.vbar(x=ts_slow, width=0.00005, 
       bottom=np.zeros_like(ts_slow), top=np.ones_like(ts_slow), 
       color="yellow", alpha=0.7)
p.xaxis.axis_label = 'Times (s)'
p.yaxis.axis_label = 'Spikes'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

p = figure(plot_width=400, plot_height=200)
p.vbar(x=ts_fast, width=0.00005, 
       bottom=np.zeros_like(ts_fast), top=np.ones_like(ts_fast), 
       color="green", alpha=0.6)
p.xaxis.axis_label = 'Times (s)'
p.yaxis.axis_label = 'Spikes'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Save

- Ref, shift, add

In [262]:
times = voltages_ref['times']
v_c = voltages_ref['V_comp'][0, :]
v_o = voltages_ref['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_fs_f20_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_ref", ns_ref, ts_ref)

In [263]:
times = voltages_slow['times']
v_c = voltages_slow['V_comp'][0, :]
v_o = voltages_slow['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_fs_4_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_shift", ns_shift, ts_shift)

In [264]:
times = voltages_fast['times']
v_c = voltages_fast['V_comp'][0, :]
v_o = voltages_fast['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_fs_f30_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_add", ns_add, ts_add)